## 1. Setup

In [10]:
import os
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import pandas as pd
from sklearn import feature_extraction, model_selection, pipeline, manifold, preprocessing


Import Cuisine Data

In [3]:
cuisine = pd.read_json('./data/train.json')

Import Recipe Data

In [4]:
all_recipes = pd.read_json('./data/recipes_raw_nosource_allrecipes.json', orient='index')
epicurious = pd.read_json('./data/recipes_raw_nosource_epicurious.json', orient='index')
food_network = pd.read_json('./data/recipes_raw_nosource_foodnetwork.json', orient='index')
recipes = pd.concat([all_recipes, epicurious, food_network], axis=0)

In [6]:
recipes = recipes.reset_index()
recipes = recipes.drop(columns=['index', 'picture_link'])

In [7]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124647 entries, 0 to 124646
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         124595 non-null  object
 1   ingredients   124647 non-null  object
 2   instructions  124473 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


## 2. Pre-Processing

### A. Cleaning and Tokenization

Add stopwords

In [9]:
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
addl_stop_words = ['advertisement', 'advertisments', 'cup', 'cups', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons', 'ounce', 'ounces', 'salt', 'pepper', 'pound', 'pounds']
stopwords.extend(addl_stop_words)


String cleaning function

In [15]:
def clean_string(list, lemmatize = True, stemming = False):
    str = ' '.join(list)
    clean_text = ''

    for text in str:
        lower = text.lower() #lowercase the words
        token = word_tokenize(lower) #tokenize words
        rem_stop = [i for i in token if i not in stopwords] #remove stopwords
        
        clean_words = []
        for word in rem_stop: 
            if len(word) > 2 and re.sub(r'\d+','', word) and re.sub(' +', ' ', word):
                #remove punctuation, digits,  and retain word length greater than 2
                
                if lemmatize:
                    lm = WordNetLemmatizer()  #lemmatize
                    lemm = lm.lemmatize(word)
                    
                    clean_words.append(lemm)

                if stemming:
                    stemmer = PorterStemmer # stem
                    stemm = stemmer.stem(word)

                    clean_words.append(stemm)

        clean_text = " ".join(clean_words)
        
    return clean_text

Clean Ingredients

In [16]:
recipes['clean_ingredients'] = clean_string(recipes['ingredients'])

TypeError: sequence item 0: expected str instance, list found